# ESM1b analysis 

#### 1) The E.coli LTEE data was retrived from the authors github page -> https://github.com/benjaminhgood/LTEE-metagenomic
#### 2) The path of the folder is data_files , from here the all population annotated_timecourse text was taken and a masterhseet was prepared with only missense mutations.
#### 3) Inorder to process this file for ESM1b analysis we need all variants WT protien sequence. We retrived the DNA and protein sequence from NCBI --> https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/017/985/GCF_000017985.1_ASM1798v1/ . 
#### 4) The NCBI files such as cds_genomics.fna and translate_cds.faa along with feature table.txt was important for this analysis.

# Step 1

#### To retrive the DNA sequence for each variants, we need to do the following steps.

In [1]:
import pandas as pd

# Step 1: Open and read the text file
file_path = "/home/bernadettem/bernadettenotebook/E.Coli_LTEE/Prt_final/esm1b_redo/Correspondance/NCBI_data/GCA_000017985.1_ASM1798v1_cds_from_genomic.fna"  # Replace with your file path
data = []

try:
    with open(file_path, 'r') as file:
        header = None
        sequence = []

        for line in file:
            if line.startswith(">"):
                # If a new header is encountered, save the previous sequence (if any)
                if header is not None:
                    data.append([header, ''.join(sequence)])
                header = line.strip()
                sequence = []
            else:
                sequence.append(line.strip())

        # Add the last sequence (if any)
        if header is not None:
            data.append([header, ''.join(sequence)])

    # Step 2: Create a DataFrame
    df = pd.DataFrame(data, columns=['col1', 'col2'])

    # Step 3: Display the DataFrame
    print(df.head())

except FileNotFoundError:
    print(f"File '{file_path}' not found.")
except IOError:
    print(f"An error occurred while reading the file '{file_path}'.")


                                                col1  \
0  >lcl|CP000819.1_cds_ACT37694.1_1 [gene=thrL] [...   
1  >lcl|CP000819.1_cds_ACT37695.1_2 [gene=thrA] [...   
2  >lcl|CP000819.1_cds_ACT37696.1_3 [gene=thrB] [...   
3  >lcl|CP000819.1_cds_ACT37697.1_4 [gene=thrC] [...   
4  >lcl|CP000819.1_cds_ACT37698.1_5 [gene=yaaX] [...   

                                                col2  
0  ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCA...  
1  ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...  
2  ATGGTTAAAGTTTATGCCCCGGCTTCCAGTGCCAATATGAGCGTCG...  
3  ATGAAACTCTACAATCTGAAAGATCACAATGAGCAGGTCAGCTTTG...  
4  ATGAAAAAGATGCAATCTATCGTACTCGCACTTTCCCTGGTTCTGG...  


In [2]:
# Process the first row to extract cols 
import pandas as pd

# Define a regular expression pattern to match attributes in square brackets
pattern = r'\[([^]]+)\]'

# Split the "col2" column by the attribute pattern and expand it into multiple columns
split_columns = df['col1'].str.split(pattern, expand=True)

# Rename the new columns for clarity (optional)
split_columns.columns = [f'Attribute_{i}' for i in range(len(split_columns.columns))]

# Concatenate the split_columns DataFrame with the original DataFrame
result_df = pd.concat([df, split_columns], axis=1)

# Drop the original "col2" column if needed
# result_df = result_df.drop(columns='col2')

# Print the result
print(result_df)


                                                   col1  \
0     >lcl|CP000819.1_cds_ACT37694.1_1 [gene=thrL] [...   
1     >lcl|CP000819.1_cds_ACT37695.1_2 [gene=thrA] [...   
2     >lcl|CP000819.1_cds_ACT37696.1_3 [gene=thrB] [...   
3     >lcl|CP000819.1_cds_ACT37697.1_4 [gene=thrC] [...   
4     >lcl|CP000819.1_cds_ACT37698.1_5 [gene=yaaX] [...   
...                                                 ...   
4204  >lcl|CP000819.1_cds_ACT41898.1_4205 [gene=creB...   
4205  >lcl|CP000819.1_cds_ACT41899.1_4206 [gene=creC...   
4206  >lcl|CP000819.1_cds_ACT41900.1_4207 [gene=arcA...   
4207  >lcl|CP000819.1_cds_ACT41901.1_4208 [gene=yjjY...   
4208  >lcl|CP000819.1_cds_ACT41902.1_4209 [gene=lasT...   

                                                   col2  \
0     ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCA...   
1     ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...   
2     ATGGTTAAAGTTTATGCCCCGGCTTCCAGTGCCAATATGAGCGTCG...   
3     ATGAAACTCTACAATCTGAAAGATCACAATGAGCAGGTCAGCTTTG...

In [3]:
# Rename columns
# Assuming you want to rename "col2" to "new_column_name"
result_df.columns = ['Info' if col == 'col1' else col for col in result_df.columns]
# # Assuming you want to rename "col2" to "new_column_name"
result_df.columns = ['DNA_Seq' if col == 'col2' else col for col in result_df.columns]
result_df.columns = ['Function' if col == 'Attribute_5' else col for col in result_df.columns]
result_df.columns = ['protein_id' if col == 'Attribute_7' else col for col in result_df.columns]
result_df.columns = ['Location' if col == 'Attribute_9' else col for col in result_df.columns]
result_df.columns = ['locus_tag' if col == 'Attribute_3' else col for col in result_df.columns]

In [4]:
# Split "col2" based on "=" and create new columns C and D
result_df[['C', 'D']] = result_df['Attribute_1'].str.split('=', n=1, expand=True)

In [5]:
result_df.columns = ['Gene' if col == 'D' else col for col in result_df.columns]

In [6]:
# Check for spaces between characters in each row of "col2"
result_df['contains_space'] = result_df['Gene'].str.contains(r'\s')

# Replace True with "Yes" and False with "No" in the "contains_
# " column
result_df['contains_space'] = result_df['contains_space'].replace({True: 'Yes', False: 'No'})

# Print the DataFrame
print(result_df)

                                                   Info  \
0     >lcl|CP000819.1_cds_ACT37694.1_1 [gene=thrL] [...   
1     >lcl|CP000819.1_cds_ACT37695.1_2 [gene=thrA] [...   
2     >lcl|CP000819.1_cds_ACT37696.1_3 [gene=thrB] [...   
3     >lcl|CP000819.1_cds_ACT37697.1_4 [gene=thrC] [...   
4     >lcl|CP000819.1_cds_ACT37698.1_5 [gene=yaaX] [...   
...                                                 ...   
4204  >lcl|CP000819.1_cds_ACT41898.1_4205 [gene=creB...   
4205  >lcl|CP000819.1_cds_ACT41899.1_4206 [gene=creC...   
4206  >lcl|CP000819.1_cds_ACT41900.1_4207 [gene=arcA...   
4207  >lcl|CP000819.1_cds_ACT41901.1_4208 [gene=yjjY...   
4208  >lcl|CP000819.1_cds_ACT41902.1_4209 [gene=lasT...   

                                                DNA_Seq  \
0     ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCA...   
1     ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...   
2     ATGGTTAAAGTTTATGCCCCGGCTTCCAGTGCCAATATGAGCGTCG...   
3     ATGAAACTCTACAATCTGAAAGATCACAATGAGCAGGTCAGCTTTG...

In [7]:
# Check if any of the character contains spaces 
result_df['contains_space'].unique()

array(['No'], dtype=object)

In [8]:
result_df.columns

Index(['Info', 'DNA_Seq', 'Attribute_0', 'Attribute_1', 'Attribute_2',
       'locus_tag', 'Attribute_4', 'Function', 'Attribute_6', 'protein_id',
       'Attribute_8', 'Location', 'Attribute_10', 'Attribute_11',
       'Attribute_12', 'Attribute_13', 'Attribute_14', 'C', 'Gene',
       'contains_space'],
      dtype='object')

In [9]:
# Select the columns you want to keep
DNA = result_df[['Info', 'DNA_Seq', 'Attribute_0', 'Attribute_1', 'Attribute_2',
       'locus_tag', 'Attribute_4', 'Function', 'Attribute_6', 'protein_id',
       'Attribute_8', 'Location', 'Attribute_10', 'Attribute_11',
       'Attribute_12', 'Attribute_13', 'Attribute_14', 'C', 'Gene',]]

In [10]:
# Write in local 
# Specify the file path where you want to save the Excel file
excel_file_path = '/home/bernadettem/bernadettenotebook/E.Coli_LTEE/Prt_final/esm1b_redo/Correspondance/Results/DNA_seq.xlsx'

# Write the DataFrame to an Excel file
DNA.to_excel(excel_file_path, index=False)

print(f'DataFrame saved to {excel_file_path}')

DataFrame saved to /home/bernadettem/bernadettenotebook/E.Coli_LTEE/Prt_final/esm1b_redo/Correspondance/DNA_seq.xlsx


#### To retrive the Protein sequence for each variants, we need to do the following steps.

In [1]:
# Read the prt file from ncbi 
import pandas as pd

# Step 1: Open and read the text file
file_path = "/home/bernadettem/bernadettenotebook/E.Coli_LTEE/Prt_final/esm1b_redo/Correspondance/NCBI_data/GCA_000017985.1_ASM1798v1_translated_cds.faa"  # Replace with your file path
data = []

try:
    with open(file_path, 'r') as file:
        header = None
        sequence = []

        for line in file:
            if line.startswith(">"):
                # If a new header is encountered, save the previous sequence (if any)
                if header is not None:
                    data.append([header, ''.join(sequence)])
                header = line.strip()
                sequence = []
            else:
                sequence.append(line.strip())

        # Add the last sequence (if any)
        if header is not None:
            data.append([header, ''.join(sequence)])

    # Step 2: Create a DataFrame
    df = pd.DataFrame(data, columns=['col1', 'col2'])

    # Step 3: Display the DataFrame
    print(df.head())

except FileNotFoundError:
    print(f"File '{file_path}' not found.")
except IOError:
    print(f"An error occurred while reading the file '{file_path}'.")



                                                col1  \
0  >lcl|CP000819.1_prot_ACT37694.1_1 [gene=thrL] ...   
1  >lcl|CP000819.1_prot_ACT37695.1_2 [gene=thrA] ...   
2  >lcl|CP000819.1_prot_ACT37696.1_3 [gene=thrB] ...   
3  >lcl|CP000819.1_prot_ACT37697.1_4 [gene=thrC] ...   
4  >lcl|CP000819.1_prot_ACT37698.1_5 [gene=yaaX] ...   

                                                col2  
0                              MKRISTTITTTITITTGNGAG  
1  MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...  
2  MVKVYAPASSANMSVGFDVLGAAVTPVDGALLGDVVTVEAAETFSL...  
3  MKLYNLKDHNEQVSFAQAVTQGLGKNQGLFFPHDLPEFSLTEIDEM...  
4  MKKMQSIVLALSLVLVAPMAAQAAEITLVPSVKLQIGDRDNRGYYW...  


In [2]:
# process the rows 
import pandas as pd

# Define a regular expression pattern to match attributes in square brackets
pattern = r'\[([^]]+)\]'

# Split the "col2" column by the attribute pattern and expand it into multiple columns
split_columns = df['col1'].str.split(pattern, expand=True)

# Rename the new columns for clarity (optional)
split_columns.columns = [f'Attribute_{i}' for i in range(len(split_columns.columns))]

# Concatenate the split_columns DataFrame with the original DataFrame
result_df = pd.concat([df, split_columns], axis=1)

# Drop the original "col2" column if needed
# result_df = result_df.drop(columns='col2')

# Print the result
print(result_df)


                                                   col1  \
0     >lcl|CP000819.1_prot_ACT37694.1_1 [gene=thrL] ...   
1     >lcl|CP000819.1_prot_ACT37695.1_2 [gene=thrA] ...   
2     >lcl|CP000819.1_prot_ACT37696.1_3 [gene=thrB] ...   
3     >lcl|CP000819.1_prot_ACT37697.1_4 [gene=thrC] ...   
4     >lcl|CP000819.1_prot_ACT37698.1_5 [gene=yaaX] ...   
...                                                 ...   
4204  >lcl|CP000819.1_prot_ACT41898.1_4205 [gene=cre...   
4205  >lcl|CP000819.1_prot_ACT41899.1_4206 [gene=cre...   
4206  >lcl|CP000819.1_prot_ACT41900.1_4207 [gene=arc...   
4207  >lcl|CP000819.1_prot_ACT41901.1_4208 [gene=yjj...   
4208  >lcl|CP000819.1_prot_ACT41902.1_4209 [gene=las...   

                                                   col2  \
0                                 MKRISTTITTTITITTGNGAG   
1     MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...   
2     MVKVYAPASSANMSVGFDVLGAAVTPVDGALLGDVVTVEAAETFSL...   
3     MKLYNLKDHNEQVSFAQAVTQGLGKNQGLFFPHDLPEFSLTEIDEM...

In [3]:
# Rename the rows 
# Assuming you want to rename "col2" to "new_column_name"
result_df.columns = ['Info' if col == 'col1' else col for col in result_df.columns]
# # Assuming you want to rename "col2" to "new_column_name"
result_df.columns = ['Prt_Seq' if col == 'col2' else col for col in result_df.columns]
result_df.columns = ['Function' if col == 'Attribute_5' else col for col in result_df.columns]
result_df.columns = ['protein_id' if col == 'Attribute_7' else col for col in result_df.columns]
result_df.columns = ['Location' if col == 'Attribute_9' else col for col in result_df.columns]
result_df.columns = ['locus_tag' if col == 'Attribute_3' else col for col in result_df.columns]

In [4]:
# Split "col2" based on "=" and create new columns C and D
result_df[['C', 'D']] = result_df['Attribute_1'].str.split('=', n=1, expand=True)

In [5]:
result_df.columns = ['Gene' if col == 'D' else col for col in result_df.columns]

In [6]:
# Check for spaces between characters in each row of "col2"
result_df['contains_space'] = result_df['Gene'].str.contains(r'\s')

# Replace True with "Yes" and False with "No" in the "contains_space" column
result_df['contains_space'] = result_df['contains_space'].replace({True: 'Yes', False: 'No'})

# Print the DataFrame
print(result_df)

                                                   Info  \
0     >lcl|CP000819.1_prot_ACT37694.1_1 [gene=thrL] ...   
1     >lcl|CP000819.1_prot_ACT37695.1_2 [gene=thrA] ...   
2     >lcl|CP000819.1_prot_ACT37696.1_3 [gene=thrB] ...   
3     >lcl|CP000819.1_prot_ACT37697.1_4 [gene=thrC] ...   
4     >lcl|CP000819.1_prot_ACT37698.1_5 [gene=yaaX] ...   
...                                                 ...   
4204  >lcl|CP000819.1_prot_ACT41898.1_4205 [gene=cre...   
4205  >lcl|CP000819.1_prot_ACT41899.1_4206 [gene=cre...   
4206  >lcl|CP000819.1_prot_ACT41900.1_4207 [gene=arc...   
4207  >lcl|CP000819.1_prot_ACT41901.1_4208 [gene=yjj...   
4208  >lcl|CP000819.1_prot_ACT41902.1_4209 [gene=las...   

                                                Prt_Seq  \
0                                 MKRISTTITTTITITTGNGAG   
1     MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...   
2     MVKVYAPASSANMSVGFDVLGAAVTPVDGALLGDVVTVEAAETFSL...   
3     MKLYNLKDHNEQVSFAQAVTQGLGKNQGLFFPHDLPEFSLTEIDEM...

In [7]:
result_df['contains_space'].unique()

array(['No'], dtype=object)

In [8]:
# Check for duplicate values in the "col2" column
duplicates = result_df[result_df['Gene'].duplicated(keep=False)]

# Print the duplicate rows
print(duplicates)

Empty DataFrame
Columns: [Info, Prt_Seq, Attribute_0, Attribute_1, Attribute_2, locus_tag, Attribute_4, Function, Attribute_6, protein_id, Attribute_8, Location, Attribute_10, Attribute_11, Attribute_12, Attribute_13, Attribute_14, C, Gene, contains_space]
Index: []


In [9]:
# Select the columns you want to keep
Prt = result_df[['Info', 'Prt_Seq', 'Attribute_0', 'Attribute_1', 'Attribute_2',
       'locus_tag', 'Attribute_4', 'Function', 'Attribute_6', 'protein_id',
       'Attribute_8', 'Location', 'Attribute_10', 'Attribute_11',
       'Attribute_12', 'Attribute_13', 'Attribute_14', 'C', 'Gene',]]

In [10]:
# Write in local 
# Specify the file path where you want to save the Excel file
excel_file_path = '/home/bernadettem/bernadettenotebook/E.Coli_LTEE/Prt_final/esm1b_redo/Correspondance/Results/Prt_seq.xlsx'

# Write the DataFrame to an Excel file
result_df.to_excel(excel_file_path, index=False)

print(f'DataFrame saved to {excel_file_path}')

DataFrame saved to /home/bernadettem/bernadettenotebook/E.Coli_LTEE/Prt_final/esm1b_redo/Correspondance/Results/Prt_seq.xlsx


#### Now we have DNA and protien sequence for each gene. So merge it with variant file.

In [12]:
import pandas as pd
df = pd.read_excel('/home/bernadettem/bernadettenotebook/E.Coli_LTEE/Prt_final/esm1b_redo/Correspondance/Mastersheet.xlsx',sheet_name='missense') #Variant sheet 
dna = pd.read_excel("/home/bernadettem/bernadettenotebook/E.Coli_LTEE/Prt_final/esm1b_redo/Correspondance/Results/DNA_seq.xlsx") # DNA sheet 
prt = pd.read_excel("/home/bernadettem/bernadettenotebook/E.Coli_LTEE/Prt_final/esm1b_redo/Correspondance/Results/Prt_seq.xlsx") # Protein sheet 

In [13]:
# Merge DNA with prt firstly then merge thus df with the original sheet 
DNA_prt = dna.merge(prt, on='Gene', how='left')

In [14]:
# Merge DNA with prt firstly then merge thus df with the original sheet 
Final_df= df.merge(DNA_prt, on='Gene', how='left')

In [16]:
# Write in local 
# Specify the file path where you want to save the Excel file
excel_file_path = '/home/bernadettem/bernadettenotebook/E.Coli_LTEE/Prt_final/esm1b_redo/Correspondance/Results/DNA_Prt_variant.xlsx'

# Write the DataFrame to an Excel file
Final_df.to_excel(excel_file_path, index=False)

print(f'DataFrame saved to {excel_file_path}')

DataFrame saved to /home/bernadettem/bernadettenotebook/E.Coli_LTEE/Prt_final/esm1b_redo/Correspondance/Results/DNA_Prt_variant.xlsx


# Step2 

#### Prepare this data as per input format for esm1b

In [17]:
# Check if U is present in any of the prt seq 
# Check if 'U' is present in any row of 'Prt_Seq'
contains_U = Final_df['Prt_Seq'].str.contains('U')

# Print the result
if contains_U.any():
    print("The letter 'U' is present in at least one row of 'Prt_Seq'.")
else:
    print("The letter 'U' is not present in any row of 'Prt_Seq'.")

The letter 'U' is present in at least one row of 'Prt_Seq'.


In [19]:
# Check if 'U' is present in any row of 'Prt_Seq'
contains_U = Final_df['Prt_Seq'].str.contains('U')

# Find the rows where 'U' is present
rows_with_U = Final_df.loc[contains_U]

# Print the rows containing 'U'
print(rows_with_U)

       Position  Gene Allele Annotation  Test.statistic   P-value Ref_allele  \
1054    1522293  fdnG   A->C   missense        1.385120  0.000100          A   
1055    1522820  fdnG   A->C   missense        3.572310  0.001300          A   
1056    1523598  fdnG   A->C   missense        2.565300  0.000100          A   
1057    1524108  fdnG   T->G   missense        2.080730  0.000100          T   
2971    4060956  fdoG   T->G   missense        2.337940  0.011356          T   
2972    4062141  fdoG   T->G   missense        2.423190  0.002700          T   
2973    4062272  fdoG   T->G   missense        1.401510  0.000100          T   
3137    4276594  fdhF   T->G   missense        2.645590  0.000200          T   
3138    4277719  fdhF   A->C   missense        1.647880  0.000100          A   
4022    1523115  fdnG   A->G   missense        3.006440  0.005999          A   
4023    1523184  fdnG   A->G   missense        1.021040  0.000300          A   
4024    1523997  fdnG   C->T   missense 

In [20]:
# Check if 'U' is present in any row of 'col2'
contains_U = Final_df['Prt_Seq'].str.contains('U')

# Create a new DataFrame without rows containing 'U'
Final_df = Final_df[~contains_U]

# Display the filtered DataFrame
print(Final_df)

       Position  Gene Allele Annotation  Test.statistic   P-value Ref_allele  \
0           241  thrL   A->C   missense        2.887050  0.000100          A   
1          1615  thrA   T->G   missense        0.986369  0.000100          T   
2          6046  yaaA   C->A   missense        1.681940  0.077982          C   
3          6220  yaaA   T->G   missense        2.478710  0.000100          T   
4          7406  yaaJ   A->C   missense        3.437140  0.002300          A   
...         ...   ...    ...        ...             ...       ...        ...   
22852   4627830  arcA   C->T   missense        1.390190  0.012235          C   
22853   4627933  arcA   T->G   missense        2.990930  0.005099          T   
22854   4628381  arcA   T->G   missense        1.179320  0.000100          T   
22855   4628701  yjjY   A->C   missense        5.263680  0.000100          A   
22856   4629600  lasT   A->G   missense        1.506350  0.000100          A   

      Alt_allele label                 

In [21]:
# Drop nan 
Final_df = Final_df.dropna(subset=['Prt_Seq'])

In [22]:
# List of columns you want to keep esm1b input 
columns_to_keep = ['Position', 'Gene','Prt_Seq']

# Select the desired columns and assign them back to the DataFrame
Final_df = Final_df[columns_to_keep]

In [23]:
# Merge 'Position' and 'Gene' into a new column 'seq_id' with an underscore separator
Final_df['seq_id'] = Final_df['Position'].astype(str) + '_' + df['Gene'].astype(str)

In [24]:
# List of columns you want to keep
columns_to_keep = ['seq_id', 'Prt_Seq']

# Select the desired columns and assign them back to the DataFrame
Final_df = Final_df[columns_to_keep]

In [25]:
Final_df.columns = ['Prt_Seq_WT' if col == 'Prt_Seq' else col for col in Final_df.columns]

In [26]:
## Esm1B format file 
import pandas as pd

# Read the Excel file
# all_data = pd.read_excel("/content/drive/MyDrive/compiled_final.xlsx")

# Output file name
output_file = '/home/bernadettem/bernadettenotebook/E.Coli_LTEE/Prt_final/esm1b_redo/Correspondance/Results/esm1b_input_WT.fasta'

# Write to the text file
with open(output_file, 'w') as f:
    for index, row in Final_df.iterrows():
        #f.write(f'>{row["seq_id"]}\n{row["aa_seq_Mut"]}\n')
        f.write(f'>{row["seq_id"]}\n{row["Prt_Seq_WT"]}\n')

print('Data written to', output_file)

Data written to /home/bernadettem/bernadettenotebook/E.Coli_LTEE/Prt_final/esm1b_redo/Correspondance/Results/esm1b_input_WT.fasta


#### This was the source of the ESM1b model -> https://github.com/ntranoslab/esm-variants and this was the command line python3 esm_score_missense_mutations.py --input-fasta-file /path/to/input.fasta --output-csv-file /path/to/output.csv


# Step 3

In [27]:
# Load the sheet before transforming it to esm1b input 
# read the sheet 
Final_df = pd.read_excel('/home/bernadettem/bernadettenotebook/E.Coli_LTEE/Prt_final/esm1b_redo/Correspondance/Results/DNA_Prt_variant.xlsx')

#### This sheet was used to get the location GCA_000017985.1_ASM1798v1_feature_table.txt and was put in the DNA_Prt_variant in excel by vlookup and all the unwanted columns were removed.Also those variants were removed which did not have gene start and stop and genes which had upstream variants.

In [28]:
import pandas as pd
Final_df = pd.read_excel('/home/bernadettem/bernadettenotebook/E.Coli_LTEE/Prt_final/esm1b_redo/Correspondance/Results/DNA_Prt_Strand_variant.xlsx')

#### Now take this file to script2.R script to generate the mutant DNA sequence.

### After generating Mut DNA seq from script2.R get the file here.
### Now convert the WT DNA to WT prt to check if it matches the NCBI submitted Prt seq

In [3]:
import pandas as pd 
data = pd.read_excel("~/bernadettenotebook/E.Coli_LTEE/Prt_final/esm1b_redo/Correspondance/Results/DNA_Prt_Strand_Variant_MutDNA.xlsx")

In [4]:
from Bio.Seq import Seq
from Bio.Data import CodonTable
import pandas as pd

# Define the custom codon table
custom_codon_table = CodonTable.unambiguous_dna_by_id[11]  # Change the ID as needed
data['WT_aa_made'] = data['DNA_Seq'].apply(lambda cds_sequence: str(Seq(cds_sequence).translate(table=custom_codon_table)).replace('*', ' '))
data

,Position,Gene,Allele,Ref_allele,Alt_allele,Annotation,label,Start,End,Strand,DNA_Seq,Prt_Seq,Sequence_Mut,Diff_Count,Differences,Diff_Pos,has_space,WT_aa_made
0,241,thrL,A->C,A,C,missense,m1,190,255,+,ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCA...,MKRISTTITTTITITTGNGAG,ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCA...,1,AC,241,False,MKRISTTITTTITITTGNGAG
1,1615,thrA,T->G,T,G,missense,m1,336,2798,+,ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...,ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...,1,TG,1615,False,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...
2,8430,talB,T->G,T,G,missense,m1,8236,9189,+,ATGACGGACAAATTGACCTCCCTTCGTCAGTACACCACCGTAGTGG...,MTDKLTSLRQYTTVVADTGDIAAMKLYQPQDATTNPSLILNAAQIP...,ATGACGGACAAATTGACCTCCCTTCGTCAGTACACCACCGTAGTGG...,1,TG,8430,False,MTDKLTSLRQYTTVVADTGDIAAMKLYQPQDATTNPSLILNAAQIP...
3,12527,dnaK,A->C,A,C,missense,m1,12161,14077,+,ATGGGTAAAATAATTGGTATCGACCTGGGTACTACCAACTCTTGTG...,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,ATGGGTAAAATAATTGGTATCGACCTGGGTACTACCAACTCTTGTG...,1,AC,12527,False,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...
4,13974,dnaK,A->C,A,C,missense,m1,12161,14077,+,ATGGGTAAAATAATTGGTATCGACCTGGGTACTACCAACTCTTGTG...,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,ATGGGTAAAATAATTGGTATCGACCTGGGTACTACCAACTCTTGTG...,1,AC,13974,False,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21765,4621500,yjjX,T->G,T,C,missense,p6,4621384,4621905,-,ATGCACCAAGTTGTCTGTGCGACCACCAATCCCGCTAAAATTCAGG...,MHQVVCATTNPAKIQAILQAFHEIFGEGSCHIASVAVESGVPEQPF...,ATGCACCAAGTTGTCTGTGCGACCACCAATCCCGCTAAAATTCAGG...,1,AC,4621789,False,MHQVVCATTNPAKIQAILQAFHEIFGEGSCHIASVAVESGVPEQPF...
21766,4623143,rob,T->G,T,C,missense,p6,4622601,4623470,-,ATGGATCAGGCCGGCATTATTCGCGACCTTTTAATCTGGCTGGAAG...,MDQAGIIRDLLIWLEGHLDQPLSLDNVAAKAGYSKWHLQRMFKDVT...,ATGGATCAGGCCGGCATTATTCGCGACCTTTTAATCTGGCTGGAAG...,1,AC,4622928,False,MDQAGIIRDLLIWLEGHLDQPLSLDNVAAKAGYSKWHLQRMFKDVT...
21767,4627830,arcA,C->T,C,A,missense,p6,4627750,4628466,-,ATGCAGACCCCGCACATTCTTATCGTTGAAAACGAGTTGGTAACAC...,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...,ATGCAGACCCCGCACATTCTTATCGTTGAAAACGAGTTGGTAACAC...,1,GA,4628386,False,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...
21768,4627933,arcA,T->G,T,C,missense,p6,4627750,4628466,-,ATGCAGACCCCGCACATTCTTATCGTTGAAAACGAGTTGGTAACAC...,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...,ATGCAGACCCCGCACATTCTTATCGTTGAAAACGAGTTGGTAACAC...,1,AC,4628283,False,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...


In [13]:
from Bio.Seq import Seq
from Bio.Data import CodonTable
import pandas as pd

# Define the custom codon table
custom_codon_table = CodonTable.unambiguous_dna_by_id[11]  # Change the ID as needed
data['Mut_aa_made'] = data['Sequence_Mut'].apply(lambda cds_sequence: str(Seq(cds_sequence).translate(table=custom_codon_table)).replace('*', ' '))
data

,Position,Gene,Allele,Ref_allele,Alt_allele,Annotation,label,Start,End,Strand,DNA_Seq,Prt_Seq,Sequence_Mut,Diff_Count,Differences,Diff_Pos,has_space,WT_aa_made,comparison,Mut_aa_made
0,241,thrL,A->C,A,C,missense,m1,190,255,+,ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCA...,MKRISTTITTTITITTGNGAG,ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCA...,1,AC,241,False,MKRISTTITTTITITTGNGAG,True,MKRISTTITTTITITTGHGAG
1,1615,thrA,T->G,T,G,missense,m1,336,2798,+,ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...,ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...,1,TG,1615,False,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...,True,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...
2,8430,talB,T->G,T,G,missense,m1,8236,9189,+,ATGACGGACAAATTGACCTCCCTTCGTCAGTACACCACCGTAGTGG...,MTDKLTSLRQYTTVVADTGDIAAMKLYQPQDATTNPSLILNAAQIP...,ATGACGGACAAATTGACCTCCCTTCGTCAGTACACCACCGTAGTGG...,1,TG,8430,False,MTDKLTSLRQYTTVVADTGDIAAMKLYQPQDATTNPSLILNAAQIP...,True,MTDKLTSLRQYTTVVADTGDIAAMKLYQPQDATTNPSLILNAAQIP...
3,12527,dnaK,A->C,A,C,missense,m1,12161,14077,+,ATGGGTAAAATAATTGGTATCGACCTGGGTACTACCAACTCTTGTG...,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,ATGGGTAAAATAATTGGTATCGACCTGGGTACTACCAACTCTTGTG...,1,AC,12527,False,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,True,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...
4,13974,dnaK,A->C,A,C,missense,m1,12161,14077,+,ATGGGTAAAATAATTGGTATCGACCTGGGTACTACCAACTCTTGTG...,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,ATGGGTAAAATAATTGGTATCGACCTGGGTACTACCAACTCTTGTG...,1,AC,13974,False,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,True,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21765,4621500,yjjX,T->G,T,C,missense,p6,4621384,4621905,-,ATGCACCAAGTTGTCTGTGCGACCACCAATCCCGCTAAAATTCAGG...,MHQVVCATTNPAKIQAILQAFHEIFGEGSCHIASVAVESGVPEQPF...,ATGCACCAAGTTGTCTGTGCGACCACCAATCCCGCTAAAATTCAGG...,1,AC,4621789,False,MHQVVCATTNPAKIQAILQAFHEIFGEGSCHIASVAVESGVPEQPF...,True,MHQVVCATTNPAKIQAILQAFHEIFGEGSCHIASVAVESGVPEQPF...
21766,4623143,rob,T->G,T,C,missense,p6,4622601,4623470,-,ATGGATCAGGCCGGCATTATTCGCGACCTTTTAATCTGGCTGGAAG...,MDQAGIIRDLLIWLEGHLDQPLSLDNVAAKAGYSKWHLQRMFKDVT...,ATGGATCAGGCCGGCATTATTCGCGACCTTTTAATCTGGCTGGAAG...,1,AC,4622928,False,MDQAGIIRDLLIWLEGHLDQPLSLDNVAAKAGYSKWHLQRMFKDVT...,True,MDQAGIIRDLLIWLEGHLDQPLSLDNVAAKAGYSKWHLQRMFKDVT...
21767,4627830,arcA,C->T,C,A,missense,p6,4627750,4628466,-,ATGCAGACCCCGCACATTCTTATCGTTGAAAACGAGTTGGTAACAC...,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...,ATGCAGACCCCGCACATTCTTATCGTTGAAAACGAGTTGGTAACAC...,1,GA,4628386,False,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...,True,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...
21768,4627933,arcA,T->G,T,C,missense,p6,4627750,4628466,-,ATGCAGACCCCGCACATTCTTATCGTTGAAAACGAGTTGGTAACAC...,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...,ATGCAGACCCCGCACATTCTTATCGTTGAAAACGAGTTGGTAACAC...,1,AC,4628283,False,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...,True,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...


In [5]:
# remove any empty spcae 
data['WT_aa_made'] = data['WT_aa_made'].str.replace(' ', '')

In [6]:
# remove any empty spcae 
data['Prt_Seq'] = data['Prt_Seq'].str.replace(' ', '')

In [7]:
# Compare sequences and create a new column
data['comparison'] = data.apply(lambda row: row['Prt_Seq'] == row['WT_aa_made'], axis=1)

In [8]:
data['comparison'].unique()

array([ True, False])

In [9]:
# Count the number of 'True' values in 'comparison'
false_count = (data['comparison'] == False).sum()

print("Number of 'False' values in comparison:", false_count)

Number of 'False' values in comparison: 2216


In [10]:
# Count the number of 'True' values in 'comparison'
true_count = (data['comparison'] == True).sum()

print("Number of 'True' values in comparison:", true_count)

Number of 'True' values in comparison: 19554


In [14]:
# Subset rows with 'True' values into a new DataFrame
true_df = data[data['comparison'] == True]

In [15]:
true_df

,Position,Gene,Allele,Ref_allele,Alt_allele,Annotation,label,Start,End,Strand,DNA_Seq,Prt_Seq,Sequence_Mut,Diff_Count,Differences,Diff_Pos,has_space,WT_aa_made,comparison,Mut_aa_made
0,241,thrL,A->C,A,C,missense,m1,190,255,+,ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCA...,MKRISTTITTTITITTGNGAG,ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCA...,1,AC,241,False,MKRISTTITTTITITTGNGAG,True,MKRISTTITTTITITTGHGAG
1,1615,thrA,T->G,T,G,missense,m1,336,2798,+,ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...,ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...,1,TG,1615,False,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...,True,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...
2,8430,talB,T->G,T,G,missense,m1,8236,9189,+,ATGACGGACAAATTGACCTCCCTTCGTCAGTACACCACCGTAGTGG...,MTDKLTSLRQYTTVVADTGDIAAMKLYQPQDATTNPSLILNAAQIP...,ATGACGGACAAATTGACCTCCCTTCGTCAGTACACCACCGTAGTGG...,1,TG,8430,False,MTDKLTSLRQYTTVVADTGDIAAMKLYQPQDATTNPSLILNAAQIP...,True,MTDKLTSLRQYTTVVADTGDIAAMKLYQPQDATTNPSLILNAAQIP...
3,12527,dnaK,A->C,A,C,missense,m1,12161,14077,+,ATGGGTAAAATAATTGGTATCGACCTGGGTACTACCAACTCTTGTG...,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,ATGGGTAAAATAATTGGTATCGACCTGGGTACTACCAACTCTTGTG...,1,AC,12527,False,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,True,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...
4,13974,dnaK,A->C,A,C,missense,m1,12161,14077,+,ATGGGTAAAATAATTGGTATCGACCTGGGTACTACCAACTCTTGTG...,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,ATGGGTAAAATAATTGGTATCGACCTGGGTACTACCAACTCTTGTG...,1,AC,13974,False,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,True,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21765,4621500,yjjX,T->G,T,C,missense,p6,4621384,4621905,-,ATGCACCAAGTTGTCTGTGCGACCACCAATCCCGCTAAAATTCAGG...,MHQVVCATTNPAKIQAILQAFHEIFGEGSCHIASVAVESGVPEQPF...,ATGCACCAAGTTGTCTGTGCGACCACCAATCCCGCTAAAATTCAGG...,1,AC,4621789,False,MHQVVCATTNPAKIQAILQAFHEIFGEGSCHIASVAVESGVPEQPF...,True,MHQVVCATTNPAKIQAILQAFHEIFGEGSCHIASVAVESGVPEQPF...
21766,4623143,rob,T->G,T,C,missense,p6,4622601,4623470,-,ATGGATCAGGCCGGCATTATTCGCGACCTTTTAATCTGGCTGGAAG...,MDQAGIIRDLLIWLEGHLDQPLSLDNVAAKAGYSKWHLQRMFKDVT...,ATGGATCAGGCCGGCATTATTCGCGACCTTTTAATCTGGCTGGAAG...,1,AC,4622928,False,MDQAGIIRDLLIWLEGHLDQPLSLDNVAAKAGYSKWHLQRMFKDVT...,True,MDQAGIIRDLLIWLEGHLDQPLSLDNVAAKAGYSKWHLQRMFKDVT...
21767,4627830,arcA,C->T,C,A,missense,p6,4627750,4628466,-,ATGCAGACCCCGCACATTCTTATCGTTGAAAACGAGTTGGTAACAC...,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...,ATGCAGACCCCGCACATTCTTATCGTTGAAAACGAGTTGGTAACAC...,1,GA,4628386,False,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...,True,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...
21768,4627933,arcA,T->G,T,C,missense,p6,4627750,4628466,-,ATGCAGACCCCGCACATTCTTATCGTTGAAAACGAGTTGGTAACAC...,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...,ATGCAGACCCCGCACATTCTTATCGTTGAAAACGAGTTGGTAACAC...,1,AC,4628283,False,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...,True,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...


### Some seq after conversion did not match the NCBI submitted seq , Therefore remove those variants 

In [16]:
# Write in local 
# Specify the file path where you want to save the Excel file
excel_file_path = '/home/bernadettem/bernadettenotebook/E.Coli_LTEE/Prt_final/esm1b_redo/Correspondance/Results/DNA_Prt_Strand_Variant_MutDNA_True.xlsx'

# Write the DataFrame to an Excel file
true_df.to_excel(excel_file_path, index=False)

print(f'DataFrame saved to {excel_file_path}')

DataFrame saved to /home/bernadettem/bernadettenotebook/E.Coli_LTEE/Prt_final/esm1b_redo/Correspondance/Results/DNA_Prt_Strand_Variant_MutDNA_True.xlsx


### Take this sheet to R to get the SNP between WT prt and Mut prt 

#### All elements in place we can pull the ESM1b score to the variant mastersheet 

In [3]:
# Read the esm1b output score 
import pandas as pd
score = pd.read_csv('/home/bernadettem/bernadettenotebook/E.Coli_LTEE/Prt_final/esm1b_redo/Correspondance/Results/esm1b_output.csv')

In [4]:
# Make universal ID 
score['universal_id'] = score['seq_id'].astype(str) + '_' + score['mut_name'].astype(str)
score['universal_id'] = score['universal_id'].str.replace(' ', '')

In [19]:
score

,seq_id,mut_name,esm_score,universal_id
0,241_thrL,M1K,-7.278638,241_thrL_M1K
1,241_thrL,M1R,-7.132564,241_thrL_M1R
2,241_thrL,M1H,-8.481606,241_thrL_M1H
3,241_thrL,M1E,-7.796053,241_thrL_M1E
4,241_thrL,M1D,-8.461714,241_thrL_M1D
...,...,...,...,...
308506095,4629600_lasT,K228M,-8.520492,4629600_lasT_K228M
308506096,4629600_lasT,K228P,-5.068690,4629600_lasT_K228P
308506097,4629600_lasT,K228Y,-7.547709,4629600_lasT_K228Y
308506098,4629600_lasT,K228F,-7.534645,4629600_lasT_K228F


In [5]:
# Read the dataframe with SNP preporcesd in r 
true_df = pd.read_excel('/home/bernadettem/bernadettenotebook/E.Coli_LTEE/Prt_final/esm1b_redo/Correspondance/Results/DNA_Prt_Strand_Variant_MutDNA_True_SNP.xlsx')

In [6]:
# Make Seq Id in mastersheet to pull scores 
true_df['seq_id'] = true_df['Position'].astype(str) + '_' + true_df['Gene'].astype(str)
true_df['universal_id'] = true_df['seq_id'].astype(str) + '_' + true_df['SNP'].astype(str)
true_df['universal_id'] = true_df['universal_id'].str.replace(' ', '')

In [7]:
true_df

,Position,Gene,Allele,Ref_allele,Alt_allele,Annotation,label,Start,End,Strand,...,Diff_Count,Differences,Diff_Pos,has_space,WT_aa_made,comparison,Mut_aa_made,SNP,seq_id,universal_id
0,241,thrL,A->C,A,C,missense,m1,190,255,+,...,1,AC,241,False,MKRISTTITTTITITTGNGAG,True,MKRISTTITTTITITTGHGAG,N18H,241_thrL,241_thrL_N18H
1,1615,thrA,T->G,T,G,missense,m1,336,2798,+,...,1,TG,1615,False,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...,True,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...,I427S,1615_thrA,1615_thrA_I427S
2,8430,talB,T->G,T,G,missense,m1,8236,9189,+,...,1,TG,8430,False,MTDKLTSLRQYTTVVADTGDIAAMKLYQPQDATTNPSLILNAAQIP...,True,MTDKLTSLRQYTTVVADTGDIAAMKLYQPQDATTNPSLILNAAQIP...,D65E,8430_talB,8430_talB_D65E
3,12527,dnaK,A->C,A,C,missense,m1,12161,14077,+,...,1,AC,12527,False,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,True,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,M123L,12527_dnaK,12527_dnaK_M123L
4,13974,dnaK,A->C,A,C,missense,m1,12161,14077,+,...,1,AC,13974,False,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,True,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,Q605P,13974_dnaK,13974_dnaK_Q605P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19549,4621500,yjjX,T->G,T,C,missense,p6,4621384,4621905,-,...,1,AC,4621789,False,MHQVVCATTNPAKIQAILQAFHEIFGEGSCHIASVAVESGVPEQPF...,True,MHQVVCATTNPAKIQAILQAFHEIFGEGSCHIASVAVESGVPEQPF...,K136Q,4621500_yjjX,4621500_yjjX_K136Q
19550,4623143,rob,T->G,T,C,missense,p6,4622601,4623470,-,...,1,AC,4622928,False,MDQAGIIRDLLIWLEGHLDQPLSLDNVAAKAGYSKWHLQRMFKDVT...,True,MDQAGIIRDLLIWLEGHLDQPLSLDNVAAKAGYSKWHLQRMFKDVT...,S110R,4623143_rob,4623143_rob_S110R
19551,4627830,arcA,C->T,C,A,missense,p6,4627750,4628466,-,...,1,GA,4628386,False,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...,True,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...,E213K,4627830_arcA,4627830_arcA_E213K
19552,4627933,arcA,T->G,T,C,missense,p6,4627750,4628466,-,...,1,AC,4628283,False,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...,True,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...,K178N,4627933_arcA,4627933_arcA_K178N


In [8]:
# Merge 
final_df = true_df.merge(score, on='universal_id', how='left')

In [9]:
final_df

,Position,Gene,Allele,Ref_allele,Alt_allele,Annotation,label,Start,End,Strand,...,has_space,WT_aa_made,comparison,Mut_aa_made,SNP,seq_id_x,universal_id,seq_id_y,mut_name,esm_score
0,241,thrL,A->C,A,C,missense,m1,190,255,+,...,False,MKRISTTITTTITITTGNGAG,True,MKRISTTITTTITITTGHGAG,N18H,241_thrL,241_thrL_N18H,241_thrL,N18H,-4.279933
1,1615,thrA,T->G,T,G,missense,m1,336,2798,+,...,False,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...,True,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...,I427S,1615_thrA,1615_thrA_I427S,1615_thrA,I427S,-11.530375
2,8430,talB,T->G,T,G,missense,m1,8236,9189,+,...,False,MTDKLTSLRQYTTVVADTGDIAAMKLYQPQDATTNPSLILNAAQIP...,True,MTDKLTSLRQYTTVVADTGDIAAMKLYQPQDATTNPSLILNAAQIP...,D65E,8430_talB,8430_talB_D65E,8430_talB,D65E,-5.331029
3,12527,dnaK,A->C,A,C,missense,m1,12161,14077,+,...,False,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,True,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,M123L,12527_dnaK,12527_dnaK_M123L,12527_dnaK,M123L,-4.669276
4,13974,dnaK,A->C,A,C,missense,m1,12161,14077,+,...,False,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,True,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,Q605P,13974_dnaK,13974_dnaK_Q605P,13974_dnaK,Q605P,-7.992450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24078,4623143,rob,T->G,T,C,missense,p6,4622601,4623470,-,...,False,MDQAGIIRDLLIWLEGHLDQPLSLDNVAAKAGYSKWHLQRMFKDVT...,True,MDQAGIIRDLLIWLEGHLDQPLSLDNVAAKAGYSKWHLQRMFKDVT...,S110R,4623143_rob,4623143_rob_S110R,4623143_rob,S110R,-5.109406
24079,4627830,arcA,C->T,C,A,missense,p6,4627750,4628466,-,...,False,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...,True,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...,E213K,4627830_arcA,4627830_arcA_E213K,4627830_arcA,E213K,-6.002205
24080,4627830,arcA,C->T,C,A,missense,p6,4627750,4628466,-,...,False,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...,True,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...,E213K,4627830_arcA,4627830_arcA_E213K,4627830_arcA,E213K,-6.002205
24081,4627933,arcA,T->G,T,C,missense,p6,4627750,4628466,-,...,False,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...,True,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...,K178N,4627933_arcA,4627933_arcA_K178N,4627933_arcA,K178N,-4.694204


In [11]:
# Count duplicate rows based on 'key_col'
duplicate_count = final_df.duplicated(subset=['universal_id']).sum()

print("Number of duplicate rows based on 'universal_id':", duplicate_count)

Number of duplicate rows based on 'universal_id': 5765


In [12]:
# Count duplicate rows based on 'universal_id'
duplicate_count = final_df.duplicated(subset=['universal_id']).sum()

# Drop duplicate rows based on 'universal_id'
final_df = final_df.drop_duplicates(subset=['universal_id'])

print("Number of duplicate rows based on 'universal_id':", duplicate_count)
print("DataFrame after dropping duplicates:")
print(final_df)

Number of duplicate rows based on 'universal_id': 5765
DataFrame after dropping duplicates:
       Position  Gene Allele Ref_allele Alt_allele Annotation label    Start  \
0           241  thrL   A->C          A          C   missense    m1      190   
1          1615  thrA   T->G          T          G   missense    m1      336   
2          8430  talB   T->G          T          G   missense    m1     8236   
3         12527  dnaK   A->C          A          C   missense    m1    12161   
4         13974  dnaK   A->C          A          C   missense    m1    12161   
...         ...   ...    ...        ...        ...        ...   ...      ...   
24076   4612040  lplA   T->G          T          C   missense    p6  4611315   
24077   4621500  yjjX   T->G          T          C   missense    p6  4621384   
24078   4623143   rob   T->G          T          C   missense    p6  4622601   
24081   4627933  arcA   T->G          T          C   missense    p6  4627750   
24082   4628381  arcA   T->G

In [13]:
# Count NaN values in the 'score' column
nan_count = final_df['esm_score'].isna().sum()

print("Number of NaN values in 'esm-score' column:", nan_count)

Number of NaN values in 'esm-score' column: 259


In [14]:
# Drop rows with NaN values in the 'score' column
final_df = final_df.dropna(subset=['esm_score'])

In [18]:
# Select the columns you want to keep
Score_missense = final_df[['Position', 'Gene', 'Allele', 'Ref_allele', 'Alt_allele', 'Annotation',
       'label', 'Start', 'End', 'Strand', 'DNA_Seq', 'Prt_Seq', 'Sequence_Mut', 'WT_aa_made','Mut_aa_made', 'SNP', 'seq_id_x', 'universal_id',
       'seq_id_y', 'mut_name', 'esm_score']]

In [19]:
Score_missense

,Position,Gene,Allele,Ref_allele,Alt_allele,Annotation,label,Start,End,Strand,...,Prt_Seq,Sequence_Mut,WT_aa_made,Mut_aa_made,SNP,seq_id_x,universal_id,seq_id_y,mut_name,esm_score
0,241,thrL,A->C,A,C,missense,m1,190,255,+,...,MKRISTTITTTITITTGNGAG,ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCA...,MKRISTTITTTITITTGNGAG,MKRISTTITTTITITTGHGAG,N18H,241_thrL,241_thrL_N18H,241_thrL,N18H,-4.279933
1,1615,thrA,T->G,T,G,missense,m1,336,2798,+,...,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...,ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...,I427S,1615_thrA,1615_thrA_I427S,1615_thrA,I427S,-11.530375
2,8430,talB,T->G,T,G,missense,m1,8236,9189,+,...,MTDKLTSLRQYTTVVADTGDIAAMKLYQPQDATTNPSLILNAAQIP...,ATGACGGACAAATTGACCTCCCTTCGTCAGTACACCACCGTAGTGG...,MTDKLTSLRQYTTVVADTGDIAAMKLYQPQDATTNPSLILNAAQIP...,MTDKLTSLRQYTTVVADTGDIAAMKLYQPQDATTNPSLILNAAQIP...,D65E,8430_talB,8430_talB_D65E,8430_talB,D65E,-5.331029
3,12527,dnaK,A->C,A,C,missense,m1,12161,14077,+,...,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,ATGGGTAAAATAATTGGTATCGACCTGGGTACTACCAACTCTTGTG...,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,M123L,12527_dnaK,12527_dnaK_M123L,12527_dnaK,M123L,-4.669276
4,13974,dnaK,A->C,A,C,missense,m1,12161,14077,+,...,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,ATGGGTAAAATAATTGGTATCGACCTGGGTACTACCAACTCTTGTG...,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,Q605P,13974_dnaK,13974_dnaK_Q605P,13974_dnaK,Q605P,-7.992450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24076,4612040,lplA,T->G,T,C,missense,p6,4611315,4612331,-,...,MSTLRLLISDSYDPWFNLAVEECIFRQMPATQRVLFLWRNADTVVI...,ATGTCCACATTACGCCTGCTCATCTCTGACTCTTACGACCCGTGGT...,MSTLRLLISDSYDPWFNLAVEECIFRQMPATQRVLFLWRNADTVVI...,MSTLRLLISDSYDPWFNLAVEECIFRQMPATQRVLFLWRNADTVVI...,K98Q,4612040_lplA,4612040_lplA_K98Q,4612040_lplA,K98Q,-8.282178
24077,4621500,yjjX,T->G,T,C,missense,p6,4621384,4621905,-,...,MHQVVCATTNPAKIQAILQAFHEIFGEGSCHIASVAVESGVPEQPF...,ATGCACCAAGTTGTCTGTGCGACCACCAATCCCGCTAAAATTCAGG...,MHQVVCATTNPAKIQAILQAFHEIFGEGSCHIASVAVESGVPEQPF...,MHQVVCATTNPAKIQAILQAFHEIFGEGSCHIASVAVESGVPEQPF...,K136Q,4621500_yjjX,4621500_yjjX_K136Q,4621500_yjjX,K136Q,-5.321062
24078,4623143,rob,T->G,T,C,missense,p6,4622601,4623470,-,...,MDQAGIIRDLLIWLEGHLDQPLSLDNVAAKAGYSKWHLQRMFKDVT...,ATGGATCAGGCCGGCATTATTCGCGACCTTTTAATCTGGCTGGAAG...,MDQAGIIRDLLIWLEGHLDQPLSLDNVAAKAGYSKWHLQRMFKDVT...,MDQAGIIRDLLIWLEGHLDQPLSLDNVAAKAGYSKWHLQRMFKDVT...,S110R,4623143_rob,4623143_rob_S110R,4623143_rob,S110R,-5.109406
24081,4627933,arcA,T->G,T,C,missense,p6,4627750,4628466,-,...,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...,ATGCAGACCCCGCACATTCTTATCGTTGAAAACGAGTTGGTAACAC...,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...,MQTPHILIVENELVTRNTLKSIFEAEGYDVFEATDGAEMHQILSEY...,K178N,4627933_arcA,4627933_arcA_K178N,4627933_arcA,K178N,-4.694204


In [20]:
# Write in local 
# Specify the file path where you want to save the Excel file
excel_file_path = '/home/bernadettem/bernadettenotebook/E.Coli_LTEE/Prt_final/esm1b_redo/Correspondance/Results/Final_esm1b_variant.xlsx'

# Write the DataFrame to an Excel file
Score_missense.to_excel(excel_file_path, index=False)

print(f'DataFrame saved to {excel_file_path}')

DataFrame saved to /home/bernadettem/bernadettenotebook/E.Coli_LTEE/Prt_final/esm1b_redo/Correspondance/Results/Final_esm1b_variant.xlsx


##### Few of them got droped out becuase of the stop codon introduced in mutant protein

#### This is the sheet having all the variants and respective esm1b score

#### The script for plot generation is in script 3.ipynb file 